### Data processing


In [3]:
import pandas as pd
df = pd.read_csv('amazon.csv')
df = df[:300]


In [4]:
df['Words'] = df['review_body'].apply(lambda x: x.split(" "))
df['Words'] = df['Words'].apply(lambda x: [word.lower() for word in x])
df['Count'] = df['review_body'].apply(lambda x: len(x.split(" ")))
df['star_rating'] = df['star_rating']-1
rows_to_delete = df[df['Count'] < 10].index
df.drop(rows_to_delete, inplace=True)

max_seq_len = df.Count.max()
max_seq_len 


553

In [5]:
class dictionary():
    def __init__(self):
        self.word2idx = {"<PAD>" : 0}
        self.idx2word = ["<PAD>"]
    
    def add(self,word):
        word_lower = word.lower()
        if word not in self.word2idx:
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            
    def __len__(self):
        return len(self.word2idx)

In [6]:
dic = dictionary()

for idx,row in df.iterrows():
    for word in row.Words:
        dic.add(word)

### dataset building

In [12]:
from  torch.utils.data import  Dataset, DataLoader
import  torch

class sentimentdataset(Dataset):
    def __init__(self,df,max_seq_len):
            self.data = df.Words.tolist()
            self.label = df.star_rating.tolist()
            self.max_seq_len= max_seq_len
    def __getitem__(self,idx):
        tokens = []
        for word in self.data[idx]:
            tokens.append(dic.word2idx[word])
            
        for  i  in  range(self.max_seq_len-len(tokens)):
            tokens.append(dic.word2idx['<PAD>'])
            
        return torch.tensor(tokens).long(),torch.tensor(self.label[idx]).long()
    def __len__(self):
        return len(self.data)

In [13]:
batch_sizes = 8

train_dataset = sentimentdataset(df,max_seq_len)
train_dataloader = DataLoader(train_dataset,batch_sizes,shuffle = True,drop_last = True)

In [14]:
test_df = df.sample(frac=0.2)
test_dataset = sentimentdataset(test_df,max_seq_len)
test_dataloader = DataLoader(test_dataset,batch_sizes,shuffle = True,drop_last = True)

### LSTM model  building


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LanguageModel(nn.Module):
    def __init__(self, max_seq_len, num_layers, hidden_dim, embedding_dim, vocab_sizes, dropout_rate=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        
        self.embedding = nn.Embedding(vocab_sizes, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            num_layers=num_layers,
                            dropout=dropout_rate,
                            batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(hidden_dim * max_seq_len, 5)  
    
    def forward(self, x, hidden, cell):
        embedding = self.embedding(x)
        output, (h, c) = self.lstm(embedding, (hidden, cell))
        y = self.dropout(output)
        y = self.flatten(y)
        y = self.fc(y)
        y = torch.softmax(y, dim=1) 
        return y
    
    def init_hidden(self, batch_sizes):
        hidden = torch.zeros(self.num_layers, batch_sizes, self.hidden_dim)
        cell = torch.zeros(self.num_layers, batch_sizes, self.hidden_dim)
        return hidden, cell


### parameter determination

In [16]:
import torch.optim as optim

num_layers = 2
hidden_dim = 512
embedding_dim = 256
vocab_sizes = len(dic) 

max_seq_len = df.Count.max()
batch_sizes = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LanguageModel(max_seq_len,num_layers,hidden_dim,embedding_dim,vocab_sizes).to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)



RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### model training

In [ ]:
def train_model(dataloader):
    model.train()
    total_loss = 0
    for idx, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        hidden, cell = model.init_hidden(batch_sizes)
        hidden = hidden.to(device)
        cell = cell.to(device)
        optimizer.zero_grad()
        
        y_pred = model(x, hidden, cell)
        print(y_pred.shape,y.shape)
        loss = F.cross_entropy(y_pred, y)  
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    TrainLoss = total_loss / len(train_dataloader)
    print("Average loss:", TrainLoss)
    
    model.eval()
    return TrainLoss

### model testing

In [ ]:

def test_model(dataloader):
    model.eval()

    total_loss = 0.
    for idx, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        hidden, cell = model.init_hidden(batch_sizes)
        hidden = hidden.to(device)
        cell = cell.to(device)
        y_pred = model(x, hidden, cell)
        loss = F.cross_entropy(y_pred, y)
        total_loss += loss.item()
        
    TestLoss = total_loss/len(test_dataloader)
    print(f"Test loss: {TestLoss}")
    return TestLoss

### model evaluation

In [10]:
import matplotlib.pyplot as plt

num_epoch = 20
train_losses = []
test_losses = []

for i in range(num_epoch):
    print(f"==== Epoch {i+1} ====")
    train_loss = train_model(train_dataloader)
    test_loss = test_model(test_dataloader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)



plt.plot(range(1, num_epoch + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epoch + 1), test_losses, label='Test Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss over Epochs')
plt.legend()

plt.show

Duplicate key in file WindowsPath('D:/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 758 ('font.family :sans-serif')
Duplicate key in file WindowsPath('D:/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 759 ('font.sans-serif :SimHei')
Duplicate key in file WindowsPath('D:/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 760 ('axes.unicode_minus :False')


==== Epoch 1 ====


NameError: name 'train_model' is not defined